In [1]:
import os
from glob import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from torch.utils.data import Dataset, DataLoader

import numpy as np
from PIL import Image

from tqdm import tqdm_notebook

In [2]:
seed = 1

lr = 0.001
momentum = 0.5

batch_size = 64
test_batch_size = 64

epochs = 5

no_cuda = False
log_interval = 100

### Model

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(5*5*50, 500)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 5*5*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

### Preprocess

In [4]:
train_path = glob('../dataset/cifar/train/*.png')[:1000]
test_path = glob('../dataset/cifar/test/*.png')[:1000]
classes = np.unique([_.split('\\')[-1].split('_')[-1].replace('.png', '') for _ in train_path])
len(train_path), len(test_path)

(1000, 1000)

In [5]:
def get_label(path):
    lbl_name = os.path.basename(path).split('_')[-1].replace('.png','')
    label = np.argmax(classes == lbl_name)
    return label

class Dataset(Dataset):
    def __init__(self, data_paths, transform=None):
        self.data_paths = data_paths
        self.transform = transform
    
    def __len__(self):
        return len(self.data_paths)
    
    def __getitem__(self, idx):
        path = self.data_paths[idx]
        
        # Read Image
        image = Image.open(path)
        
         # Get Label
        label = get_label(path)
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

In [6]:
torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

kwargs = {'num_workers' : 1, 'pin_memory' : True} if use_cuda else {}

In [7]:
train_loader = torch.utils.data.DataLoader(
    Dataset(train_path,
            transforms.Compose([
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean = [0.406],
                    std = [0.225]
                )
            ])),
    batch_size=batch_size,
    shuffle=True,
    **kwargs
)

test_loader = torch.utils.data.DataLoader(
    Dataset(test_path,
            transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(
                    mean = [0.406],
                    std = [0.225]
                )
            ])),
    batch_size=batch_size,
    shuffle=False,
    **kwargs
)

### Optimizer

In [8]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(),
                      lr=lr,
                      momentum=momentum)

### Training

In [9]:
for epoch in range(1, 6):
    # Train mode
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() # backpropagation 계산 전 0으로 기울기 초기화
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch : {} [{} / {} ({:.0f}%)]\tLoss : {:.6f}'.format(
                epoch, batch_idx*len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()
            ))
            
    # Test mode
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    
    print('\nTest set : Average loss : {:.4f}, Accuracy : {}/{} ({:.0f}%) lr : {}\n'.format(
        test_loss, correct, len(test_loader.dataset), accuracy, optimizer.param_groups[0]['lr']
    ))

Train Epoch : 1 [0 / 1000 (0%)]	Loss : 2.296892

Test set : Average loss : 2.3021, Accuracy : 77/1000 (8%) lr : 0.001

Train Epoch : 2 [0 / 1000 (0%)]	Loss : 2.287021

Test set : Average loss : 2.2997, Accuracy : 80/1000 (8%) lr : 0.001

Train Epoch : 3 [0 / 1000 (0%)]	Loss : 2.298267

Test set : Average loss : 2.2973, Accuracy : 93/1000 (9%) lr : 0.001

Train Epoch : 4 [0 / 1000 (0%)]	Loss : 2.296385

Test set : Average loss : 2.2949, Accuracy : 102/1000 (10%) lr : 0.001

Train Epoch : 5 [0 / 1000 (0%)]	Loss : 2.282809

Test set : Average loss : 2.2925, Accuracy : 119/1000 (12%) lr : 0.001



### Save Model

In [17]:
# weight만 저장
save_path = 'model_weight.pt'

torch.save(model.state_dict(), save_path)

In [18]:
# load model
model = Net().to(device)

weight_dict = torch.load(save_path)
model.load_state_dict(weight_dict)

<All keys matched successfully>

### Save Entire Model

In [19]:
save_path = 'model.pt'

torch.save(model, save_path)

In [20]:
model = torch.load(save_path)
model.eval()

Net(
  (conv1): Conv2d(3, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=1250, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

### Save, Load and Resuming Trainning

In [21]:
checkpoint_path = 'checkpoint.pt'

torch.save({
    'epoch' : epoch,
    'model_state_dict' : model.state_dict(),
    'optimizer_state_dict' : optimizer.state_dict(),
    'loss' : loss
}, checkpoint_path)

In [24]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(),
                      lr=lr,
                      momentum=momentum)

In [25]:
checkpoint = torch.load(checkpoint_path)

model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

In [26]:
model

Net(
  (conv1): Conv2d(3, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=1250, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [27]:
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0.5
    nesterov: False
    weight_decay: 0
)

In [28]:
epoch, loss

(5, tensor(2.2739, requires_grad=True))

In [29]:
for epoch in range(6, 11):
    # Train mode
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() # backpropagation 계산 전 0으로 기울기 초기화
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch : {} [{} / {} ({:.0f}%)]\tLoss : {:.6f}'.format(
                epoch, batch_idx*len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()
            ))
            
    # Test mode
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    
    print('\nTest set : Average loss : {:.4f}, Accuracy : {}/{} ({:.0f}%) lr : {}\n'.format(
        test_loss, correct, len(test_loader.dataset), accuracy, optimizer.param_groups[0]['lr']
    ))

Train Epoch : 6 [0 / 1000 (0%)]	Loss : 2.295495

Test set : Average loss : 2.2900, Accuracy : 127/1000 (13%) lr : 0.001

Train Epoch : 7 [0 / 1000 (0%)]	Loss : 2.279306

Test set : Average loss : 2.2875, Accuracy : 137/1000 (14%) lr : 0.001

Train Epoch : 8 [0 / 1000 (0%)]	Loss : 2.290500

Test set : Average loss : 2.2849, Accuracy : 140/1000 (14%) lr : 0.001

Train Epoch : 9 [0 / 1000 (0%)]	Loss : 2.283267

Test set : Average loss : 2.2821, Accuracy : 148/1000 (15%) lr : 0.001

Train Epoch : 10 [0 / 1000 (0%)]	Loss : 2.276965

Test set : Average loss : 2.2792, Accuracy : 151/1000 (15%) lr : 0.001

